# NMRPy quickstart tutorial

Refer to https://nmrpy.readthedocs.io/en/latest/quickstart.html

In [ ]:
import matplotlib
%matplotlib widget
import nmrpy
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import sysconfig
import os

## Importing

The basic NMR project object used in NMRPy is the `FidArray`, which consists of a set of `Fid` objects, each representing a single spectrum in an array of spectra.

The simplest way to instantiate an `FidArray` is by using the `from_path()` method, and specifying the path of the .fid directory:

In [ ]:
fname = os.path.join(sysconfig.get_paths()['purelib'], 'nmrpy','tests','test_data','test1.fid')
fid_array = nmrpy.from_path(fname)

You will notice that the `fid_array` object is instantiated and now owns several attributes, most of which are of the form `fidXX` where XX is a number starting at 00. These are the individual arrayed `Fid` objects.

## Apodisation and Fourier transform

To quickly visualise the imported data, we can use the plotting functions owned by each `Fid` instance. This will not display the imaginary portion of the data:

In [ ]:
fid_array.fid00.plot_ppm()

We now perform apodisation of the FIDs using the default value of 5 Hz, and visualise the result:

In [ ]:
fid_array.emhz_fids()
fid_array.fid00.plot_ppm()

Finally, we zero-fill and Fourier-transform the data into the frequency domain:

In [ ]:
fid_array.zf_fids()
fid_array.ft_fids()
fid_array.fid00.plot_ppm()

## Phase-correction

It is clear from the data visualisation that at this stage the spectra require phase-correction. NMRPy provides a number of GUI widgets for manual processing of data. In this case we will use the `phaser()` method on `fid00`.

Dragging with the left mouse button and right mouse button will apply zero- and first-order phase-correction, respectively. The cumulative phase correction for the zero-order (`p0`) and first-order (`p1`) phase angles is displayed at the bottom of the plot so that these can be applied programatically to all `Fid` objects in the `FidArray` using the `ps_fids()` method.

In [ ]:
fid_array.fid00.phaser()

Alternatively, *automatic* phase-correction can be applied at either the `FidArray` or `Fid` level. We will apply it to the whole array:

In [ ]:
fid_array.phase_correct_fids()

And plot an array of the phase-corrected data:

In [ ]:
fid_array.plot_array()

Zooming in on the relevant peaks, changing the view perspective, and filling the spectra produces a more interesting plot:

In [ ]:
fid_array.plot_array(upper_ppm=7, lower_ppm=-1, filled=True, azim=-76, elev=23)

At this stage it is useful to discard the imaginary component of our data, and possibly normalise the data (by the maximum data value amongst the `Fid` objects):

In [ ]:
fid_array.real_fids()
fid_array.norm_fids()

## Calibration

The spectra may need calibration by assigning a chemical shift to a reference peak of a known standard and adjusting the spectral offset accordingly. To this end, a `calibrate()` convenience method exists that allows the user to easily select a peak and specify the PPM. This method can be applied at either the `FidArray` or `Fid` level. We will apply it to the whole array.

Left-clicking selects a peak and its current ppm value is displayed below the spectrum. The new ppm value can be entered in a text box, and hitting `Enter` completes the calibration process. Here we have chosen triethyl phosphate (TEP) as reference peak and assigned its chemical shift value of 0.44 ppm (the original value was 0.57 ppm, and the offset of all the spectra in the array has been adjusted by 0.13 ppm after the calibration).

In [ ]:
fid_array.calibrate()

## Peak-picking

To begin the process of integrating peaks by deconvolution, we will need to pick some peaks. The `peaks` attribute of a `Fid` is an array of peak positions, and `ranges` is an array of range boundaries. These two objects are used in deconvolution to integrate the data by fitting Lorentzian/Gaussian peak shapes to the spectra. `peaks` and `ranges` may be specified programatically, or picked using the interactive GUI widget.

Left-clicking specifies a peak selection with a vertical red line. Dragging with a right-click specifies a range to fit independently with a grey rectangle. Inadvertent wrongly selected peaks can be deleted with Ctrl+left-click; wrongly selected ranges can be deleted with Ctrl+right-click. Once you are done selecting peaks and ranges, these need to be assigned to the `FidArray`; this is achieved with a Ctrl+Alt+right-click.

In [ ]:
fid_array.peakpicker(fid_number=10, assign_only_to_index=False)

Ranges divide the data into smaller portions, which significantly speeds up the process of fitting of peakshapes to the data. Range-specification also prevents incorrect peaks from being fitted by the fitting algorithm.

Having used the `peakpicker()` `FidArray` method (as opposed to the `peakpicker()` on each individual `Fid` instance), the peak and range selections have now been assigned to each Fid in the array:

In [ ]:
print(fid_array.fid00.peaks)

In [ ]:
print(fid_array.fid00.ranges)

### Peak-picking trace selector

Sometimes peaks are subject to drift so that the chemical shift changes over time; this can happen, e.g., when the pH of the reaction mixture changes as the reaction proceeds. NMRPy offers a convenient trace selector, `peakpicker_traces()`, with which the drift of the peaks can be traced over time and the chemical shift selected accordingly as appropriate for the particular `Fid`.

As for the `peakpicker()`, ranges are selected by dragging the right mouse button and can be deleted with Ctrl+right-click. A peak trace is initiated by left-clicking below the peak underneath the first `Fid` in the series. This selects a point and anchors the trace line, which is displayed in red as the mouse is moved. The trace will attempt to follow the highest peak. Further trace points can be added by repeated left-clicking, thus tracing the peak through the individual `Fid`s in the series. It is not necessary to add an anchor point for every `Fid`, only when the trace needs to change direction. Once the trace has traversed all the `Fid`s, select a final trace point (left-click) and then finalize the trace with a right-click. The trace will change colour from red to blue to indicate that it has been finalized.

Additional peaks can then be selected by initiating a new trace. Wrongly selected traces can be deleted by Ctrl+left-click at the bottom of the trace that should be removed. Note that the interactive buttons on the matplotlib toolbar for the figure can be used to zoom and pan into a region of interest of the spectra. As previously, peaks and ranges need to be assigned to the `FidArray` with Ctrl+Alt+right-click. 

In [ ]:
fid_array.peakpicker_traces(voff=0.08)

If these trace lines don't run exactly vertically, individual peaks have different chemical shifts for the different `Fid`s, although in this particular case the drift in the spectra is not significant so that `peakpicker_traces()` need not have been used and `peakpicker()` would have been sufficient. This is merely for illustrative purposes.

In [ ]:
print(fid_array.fid00.peaks)
print(fid_array.fid10.peaks)
print(fid_array.fid20.peaks)

## Deconvolution

**Note:**

If you *have not* assigned peaks and ranges using any of the peakpicker widgets above,
**uncomment the following cell** to assign peaks and ranges to continue with the deconvolution.

In [ ]:
# peaks = [ 4.73,  4.63,  4.15,  0.55]
# ranges = [[ 5.92,  3.24], [ 1.19, -0.01]]
# for fid in fid_array.get_fids():
#     fid.peaks = peaks
#     fid.ranges = ranges

Individual `Fid` objects can be deconvoluted with `deconv()`. `FidArray` objects can be deconvoluted with `deconv_fids()`. By default this is a multiprocessed method (`mp=True`), which will fit pure Lorentzian lineshapes (`frac_gauss=0.0`) to the `peaks` and `ranges` specified in each `Fid`.

We shall fit the whole array at once:

In [ ]:
fid_array.deconv_fids()

And visualise the deconvoluted spectra:

In [ ]:
fid_array.fid10.plot_deconv()

Zooming-in to a set of peaks makes clear the fitting result:

In [ ]:
fid_array.fid10.plot_deconv(upper_ppm=5.5, lower_ppm=3.5)
fid_array.fid10.plot_deconv(upper_ppm=0.9, lower_ppm=0.2)

    *Black*: original data; 
    *Blue*:  individual peak shapes (and peak numbers above); 
    *Red*:   summed peak shapes; 
    *Green*: residual (original data - summed peakshapes).

In this case, peaks 0 and 1 belong to glucose-6-phosphate, peak 2 belongs to fructose-6-phosphate, and peak 3 belongs to triethyl-phosphate (internal standard).

We can view the deconvolution result for the whole array using `plot_deconv_array()`. Fitted peaks appear in red:

In [ ]:
fid_array.plot_deconv_array(upper_ppm=6, lower_ppm=3)

Peak integrals of the complete `FidArray` are stored in `deconvoluted_integrals`, or in each individual `Fid` as `deconvoluted_integrals`.

The species integrals can easily be plotted using the following code:
    

In [ ]:
integrals = fid_array.deconvoluted_integrals.transpose()

g6p = integrals[0] + integrals[1]
f6p = integrals[2]
tep = integrals[3]

#scale species by internal standard tep (5 mM)
g6p = 5.0*g6p/tep.mean()
f6p = 5.0*f6p/tep.mean()
tep = 5.0*tep/tep.mean()

species = {'g6p': g6p,
           'f6p': f6p,
           'tep': tep}

fig = plt.figure()
ax = fig.add_subplot(111)
for k, v in species.items():
    ax.plot(fid_array.t, v, label=k)

ax.set_xlabel('min')
ax.set_ylabel('mM')
ax.legend(loc=0, frameon=False)

## Saving / Loading

The current state of any `FidArray` object can be saved to file using the `save_to_file()` method:

In [ ]:
fid_array.save_to_file(filename='fidarray.nmrpy')

And reloaded using `from_path()`:

In [ ]:
fid_array = nmrpy.data_objects.FidArray.from_path(fid_path='fidarray.nmrpy')